In this Notebook I overview dataset https://www.kaggle.com/aquib5559/amazon-ml-engineer-hiring visualize it and build a simple model to approach task of binary classification.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score


First upload dataset to the dataframe and print the shape.

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# test.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('../input/amazon-ml-engineer-hiring/train.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'test.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
df1.head()

Dataset contains 12 columns, 4 of them are catergorical and other 8 are continous floats. Last column is a target value which means, that we are going to predict it, using other 11 columns. And first one is usless since it contains a user_id which does not contain any useful information.

To approach task, first we need to visualize data to get an understending of what we are daling with.
First we need to visualize the distribution of categorical variable.

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20, 6))
data_q1 = df1['customer_active_segment'].value_counts().sort_index()
ax[0].bar(data_q1.index, data_q1, width=0.55, 
       edgecolor='gray', color='#F4B8DD',
       linewidth=0.7)

for i in data_q1.index:
    ax[0].annotate(f"{data_q1[i]}", 
                   xy=(i, data_q1[i] + 30),
                   va = 'center', ha='center',fontweight='light', fontfamily='serif',
                   color='#311D34')


for s in ['top', 'left', 'right']:
    ax[0].spines[s].set_visible(False)

ax[0].set_ylim(0, 600)    
ax[0].set_xticklabels(data_q1.index, fontfamily='serif')
ax[0].set_yticklabels(np.arange(0, 700, 100),fontfamily='serif')
fig.text(0.1, 0.95, 'customer_active_segment distribution', fontsize=18, fontweight='bold', fontfamily='serif')    
ax[0].grid(axis='y', linestyle='-', alpha=0.4)

data_x1 = df1['X1'].value_counts().sort_index()
ax[1].bar(data_x1.index, data_x1, width=0.55,
         edgecolor='gray', color='#A9EFDA', linewidth=0.7)
for i in data_x1.index:
    ax[1].annotate(f"{data_x1[i]}", 
                   xy=(i, data_x1[i] + 30),
                   va = 'center', ha='center',fontweight='light', fontfamily='serif',
                   color='#0D4534')
for s in ['top', 'left', 'right']:
    ax[1].spines[s].set_visible(False)

ax[1].set_ylim(0, 600)    
ax[1].set_xticklabels(data_q1.index, fontfamily='serif')
ax[1].set_yticklabels(np.arange(0, 700, 100),fontfamily='serif')
ax[1].text(-1.8, 650, 'X1 distribution', fontsize=18, fontweight='bold', fontfamily='serif')    
ax[1].grid(axis='y', linestyle='-', alpha=0.4)
plt.show()

Now we need to visualize continuous variables, so we get an idea about distribution of different variables, depends of the value of target value. We use a boxplot visualisation so we will also see how many outliers data contains.

In [ ]:
import seaborn as sns
pd.set_option('mode.chained_assignment', None)
col = list(df1.columns)[1:9]
df = pd.DataFrame(np.nan, index=range(8000), columns = ['Group', 'Value','target_var'])
fig, ax = plt.subplots(figsize=(20,8))

for column in col:
    for i in df1.index:
        df['Group'][i+col.index(column)*1000] = column
        #print(df)
        df['Value'][i+col.index(column)*1000] = df1[column][i]
        df['target_var'][i+col.index(column)*1000] = df1['customer_category'][i]
my_pal = {0: "#A9EFDA", 1: "#F4B8DD"}
ax.grid(axis='y', linestyle='-', alpha=0.4)
sns.boxplot(ax=ax, x='Group', y='Value', hue='target_var', data=df, palette=my_pal)
#sns.plt.show()

Not very informative and we are clearly suffering from outliers in the data. We need to clean up the data and normalize them!

In [ ]:
for column in col:
    for i in df1.index:
        df['Value'][i+col.index(column)*1000] = (df1[column][i]-min(df1[column]))/(max(df1[column])-min(df1[column]))
        df1[column][i]=(df1[column][i]-min(df1[column]))/(max(df1[column])-min(df1[column]))
        
fig, ax = plt.subplots(figsize=(20,8))
sns.boxplot(ax=ax, x='Group', y='Value', hue='target_var', data=df, palette=my_pal)
ax.grid(axis='y', linestyle='-', alpha=0.4)
plt.show()

Much better. Now we are able to see that some of variables are highly correlated with target value which will give us an ability to perform a binary classification. Number of outliers are manageble so we not gonna clean them.

And finally: hof big is every fraction of target value?

In [ ]:
df1['customer_category'].unique()
second = sum(df1['customer_category'])/len(df1['customer_category'])*100

In [ ]:
labels = ['False', 'True']
sizes = [100-second, second]
explode = (0, 0.1) 

fig1, ax1 = plt.subplots(figsize=(20, 6))
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90, colors=['#98EBD2','#F4B8DD'], textprops={'fontfamily':'serif','fontsize':14}, wedgeprops={'alpha':0.8})
ax1.axis('equal') 
ax1.text(-1.4,1.6,"Target variable customer_category", fontsize=18, fontfamily='serif', fontweight='bold')

plt.show()

We are ready to perform classification but we need to do something before. Categorical variables per se could not be used in classification. We need to decode them as integers.

In [ ]:
df1.customer_active_segment = pd.Categorical(df1.customer_active_segment)
df1.X1 = pd.Categorical(df1.X1)

df1['X1'] = df1.X1.cat.codes
df1['customer_active_segment'] = df1.customer_active_segment.cat.codes
df1.head()

Now every categorical value is replaced with coresponded integer. Last thing we need to do on the data is checking if data contains nissing values and replace them with mean value of column so thay doesn't influence a classification.

In [ ]:
df1.isnull().sum()

In [ ]:
df1 = df1.fillna(df1.mean())

Split dataset to train and test and fit the RandomForestClassifier model

In [ ]:
x = df1.iloc[:,1:len(df1.columns)-1]
y = df1.iloc[:,len(df1.columns)-1]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.33,random_state=42)

In [ ]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)


#-----Upsampling----
from sklearn.utils import resample
from collections import Counter

print("Before Upsampling:-")
print(Counter(y_train))

# X_train_upsampled, y_train_upsampled = resample(x_train[y_train == 1],
#                                                 y_train[y_train == 1],
#                                                 replace=True,
#                                                 n_samples=x_train[y_train == 0].shape[0],
#                                                 random_state=123)


# Let's use SMOTE to oversample
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
x_train_upsampled, y_train_upsampled = oversample.fit_resample(x_train,y_train)

print("After Upsampling:-")
print(Counter(y_train_upsampled))

In [ ]:
print("\n\n\n\n AFTER UPSAMPLING\n\n")
classifier = RandomForestClassifier(n_estimators = 50, random_state = 0)
classifier.fit(x_train_upsampled, y_train_upsampled)
# Predicting result for training set and validation set
predict_val_rf = classifier.predict(x_test)


# Model Performance

print("Accuracy : ", accuracy_score(y_test, predict_val_rf) *  100)
print("Recall : ", recall_score(y_test, predict_val_rf) *  100)
print("Precision : ", precision_score(y_test, predict_val_rf) *  100)
print(confusion_matrix(y_test, predict_val_rf))
print(classification_report(y_test, predict_val_rf))

In [ ]:
df1.head()

In [ ]:
fig, ax = plt.subplots(figsize =(20,8))
colors=['#98EBD2','#F4B8DD']
for i in range(2):
    x = []
    y = []
    for a in df1.index:
        if df1['customer_category'][a] == i:
            x.append(df1['customer_order_score'][a])
            y.append(df1['customer_stay_score'][a])
    #scale = 200.0 * np.random.rand(n)
    ax.scatter(x, y, c=colors[i], label=colors[i],
                   alpha=1, edgecolors='none')

ax.legend()
ax.grid(True)

plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

cols = df1.columns[1:len(df1.columns)-1]
X = df1[cols]
y = df1['customer_category']
# Build a logreg and compute the feature importances
model = LogisticRegression()
# create the RFE model and select 8 attributes
rfe = RFE(model, 8)
rfe = rfe.fit(X, y)
# summarize the selection of the attributes
print('Selected features: %s' % list(X.columns[rfe.support_]))

In [ ]:
from sklearn.feature_selection import RFECV

rfecv = RFECV(estimator=LogisticRegression(), step=1, cv=10, scoring='accuracy')
rfecv.fit(X, y)

print("Optimal number of features: %d" % rfecv.n_features_)
print('Selected features: %s' % list(X.columns[rfecv.support_]))

# Plot number of features VS. cross-validation scores
plt.figure(figsize=(10,6))
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
Selected = ['customer_visit_score', 'customer_product_search_score', 'customer_ctr_score', 'customer_stay_score', 'customer_product_variation_score', 'customer_order_score', 'customer_affinity_score']
X = df1[Selected]

plt.subplots(figsize=(8, 5))
sns.heatmap(X.corr(), annot=True, cmap="RdYlGn")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss

# create X (features) and y (response)
X = df1[Selected]
y = df1['customer_category']

# use train/test split with different random_state values
# we can change the random_state values that changes the accuracy scores
# the scores change a lot, this is why testing scores is a high-variance estimate
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# check classification scores of logistic regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]
[fpr, tpr, thr] = roc_curve(y_test, y_pred_proba)
print('Train/Test split results:')
print(logreg.__class__.__name__+" accuracy is %2.3f" % accuracy_score(y_test, y_pred))
print(logreg.__class__.__name__+" log_loss is %2.3f" % log_loss(y_test, y_pred_proba))
print(logreg.__class__.__name__+" auc is %2.3f" % auc(fpr, tpr))

idx = np.min(np.where(tpr > 0.95)) # index of the first threshold for which the sensibility > 0.95

plt.figure()
plt.plot(fpr, tpr, color='coral', label='ROC curve (area = %0.3f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot([0,fpr[idx]], [tpr[idx],tpr[idx]], 'k--', color='blue')
plt.plot([fpr[idx],fpr[idx]], [0,tpr[idx]], 'k--', color='blue')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - specificity)', fontsize=14)
plt.ylabel('True Positive Rate (recall)', fontsize=14)
plt.title('Receiver operating characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()

print("Using a threshold of %.3f " % thr[idx] + "guarantees a sensitivity of %.3f " % tpr[idx] +  
      "and a specificity of %.3f" % (1-fpr[idx]) + 
      ", i.e. a false positive rate of %.2f%%." % (np.array(fpr[idx])*100))